# Setup

In [1]:
!pip install --quiet gradio==3.44.4 transformers[sentencepiece] git+https://github.com/huggingface/peft.git typing_extensions==4.7.1 scipy datasets evaluate sacrebleu cchardet

In [2]:
import torch
import gc

# Function to clear GPU memory
def clear_gpu_memory():
    # Delete model and tensors if they are defined
    global model, inputs, input_ids
    if 'model' in globals():
        del model
    if 'inputs' in globals():
        del inputs
    if 'input_ids' in globals():
        del input_ids

    # Clear PyTorch cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Force Python's garbage collector to run
    gc.collect()

# Call the function to clear GPU memory
clear_gpu_memory()

In [4]:
import os
import sys

import transformers
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

import evaluate
from datasets import load_dataset


"""
Helpers to support streaming generate output.
Borrowed from https://github.com/oobabooga/text-generation-webui/blob/ad37f396fc8bcbab90e11ecf17c56c97bfbd4a9c/modules/callbacks.py
"""

import gc
import traceback
from queue import Queue
from threading import Thread

import torch
import transformers

class Stream(transformers.StoppingCriteria):
    def __init__(self, callback_func=None):
        self.callback_func = callback_func

    def __call__(self, input_ids, scores) -> bool:
        if self.callback_func is not None:
            self.callback_func(input_ids[0])
        return False

class Iteratorize:
    """
    Transforms a function that takes a callback
    into a lazy iterator (generator).
    """

    def __init__(self, func, kwargs={}, callback=None):
        self.mfunc = func
        self.c_callback = callback
        self.q = Queue()
        self.sentinel = object()
        self.kwargs = kwargs
        self.stop_now = False

        def _callback(val):
            if self.stop_now:
                raise ValueError
            self.q.put(val)

        def gentask():
            try:
                ret = self.mfunc(callback=_callback, **self.kwargs)
            except ValueError:
                pass
            except:
                traceback.print_exc()
                pass

            self.q.put(self.sentinel)
            if self.c_callback:
                self.c_callback(ret)

        self.thread = Thread(target=gentask)
        self.thread.start()

    def __iter__(self):
        return self

    def __next__(self):
        obj = self.q.get(True, None)
        if obj is self.sentinel:
            raise StopIteration
        else:
            return obj

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.stop_now = True

"""
A dedicated helper to manage templates and prompt building.
"""

import json
import os.path as osp
from typing import Union


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        template_name = "alpaca"
        self.template = {
            "description": "Template used by Alpaca-LoRA.",
            "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
            "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
            "response_split": "### Response:"
        }
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()


if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:  # noqa: E722
    pass

#! Check the number of available GPUs and prepare a device map
num_gpus = torch.cuda.device_count()
device_map = {i: f'cuda:{i}' for i in range(num_gpus)}


base_model = 'openthaigpt/openthaigpt-1.0.0-beta-7b-chat-ckpt-hf'
lora_weights = None
load_8bit = False
prompt_template = ""
server_name = "0.0.0.0"
share_gradio = True

prompter = Prompter(prompt_template)
tokenizer = LlamaTokenizer.from_pretrained(base_model)
if device == "cuda":
    print("DEVICEEEEE CUDAAAAA",device_map)
    model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,
        # device_map=device_map,
        device_map="auto",
    )
    if lora_weights:
      model = PeftModel.from_pretrained(
          model,
          lora_weights,
          torch_dtype=torch.float16,
      )
elif device == "mps":
    model = LlamaForCausalLM.from_pretrained(
        base_model,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
    if lora_weights:
      model = PeftModel.from_pretrained(
          model,
          lora_weights,
          device_map={"": device},
          torch_dtype=torch.float16,
      )
else:
    model = LlamaForCausalLM.from_pretrained(
        base_model, device_map={"": device}, low_cpu_mem_usage=True
    )
    if lora_weights:
      model = PeftModel.from_pretrained(
          model,
          lora_weights,
          device_map={"": device},
      )

# unwind broken decapoda-research config
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

# if not load_8bit:
#     model.half()  # seems to fix bugs for some users.

model.eval()
if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)
print("MODEL READY LA")

DEVICEEEEE CUDAAAAA {0: 'cuda:0', 1: 'cuda:1', 2: 'cuda:2', 3: 'cuda:3'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MODEL READY LA


สามารถ Click Link: https:// xxxx .gradio.live ที่ปรากฎด้านบน เพื่อเปิด New Tab ใช้แบบเต็มจอ

# Evaluate by calling a function

Streaming yield better result, 

## Init Evaluation Function

In [5]:
def text_evaluate(
        instruction,
        input=None,
        temperature=0.85,
        top_p=0.85,
        top_k=20,
        num_beams=4,
        max_new_tokens=512,
        stream_output=True,
        repetition_penalty=1,
        no_repeat_ngram=2,
        **kwargs,
    ):
    prompt = prompter.generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        do_sample=True,
        **kwargs,
    )
    generate_params = {
        "input_ids": input_ids,
        "generation_config": generation_config,
        "return_dict_in_generate": True,
        "output_scores": True,
        "max_new_tokens": max_new_tokens,
        "early_stopping": True,
        "repetition_penalty":repetition_penalty,
        "no_repeat_ngram_size":no_repeat_ngram
    }
    if stream_output:
        # Stream the reply 1 token at a time.
        # This is based on the trick of using 'stopping_criteria' to create an iterator,
        # from https://github.com/oobabooga/text-generation-webui/blob/ad37f396fc8bcbab90e11ecf17c56c97bfbd4a9c/modules/text_generation.py#L216-L243.

        def generate_with_callback(callback=None, **kwargs):
            kwargs.setdefault(
                "stopping_criteria", transformers.StoppingCriteriaList()
            )
            kwargs["stopping_criteria"].append(
                Stream(callback_func=callback)
            )
            with torch.no_grad():
                model.generate(**kwargs)

        def generate_with_streaming(**kwargs):
            return Iteratorize(
                generate_with_callback, kwargs, callback=None
            )

        with generate_with_streaming(**generate_params) as generator:
            temp = []
            for output in generator:
                # new_tokens = len(output) - len(input_ids[0])
                decoded_output = tokenizer.decode(output)

                if output[-1] in [tokenizer.eos_token_id]:
                    break
                temp.append(decoded_output)

                
        return  temp[-1]# early return for stream_output

    # Without streaming
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    return tokenizer.decode(s).split("### Response:")[1].strip()

## Prompting and Running

### Translation Thai to English 

In [6]:

instruction = "แปลภาษาไทยเป็นอังกฤษ"
print("#Instruction:", instruction, "\n")
for input in [
    "ลดน้ำหนักต้องทำอย่างไร",
    "เขียนโปรแกรม python export csv pandas",
    "วิธีการทำน้ำจิ้มไก่",
    "แต่งกลอนวันแม่",
    "ศึกชิงนางเดือด สามีแทงกิ๊กหนุ่มเสียชีวิตกลางถนนเพชรบุรีตัดใหม่ ก่อนหน้านี้ เคยปะทะคารมกันมาแล้วไม่มีใครยอมใคร",
    "กรุงเทพมหานคร เป็นเมืองหลวงและนครที่มีประชากรมากที่สุดของประเทศไทย เป็นศูนย์กลางการปกครอง การศึกษา การคมนาคมขนส่ง การเงินการธนาคาร การพาณิชย์ การสื่อสาร และความเจริญของประเทศ"
    ""
    
]:
    def prompt(input="hello motherfucker", instruction="แปลภาษาไทยเป็นอังกฤษ", prnout=False):
        out = text_evaluate(instruction=instruction, 
                                         input=input,                                         
                                        )
        if prnout:
            for i in out.split("###")[-2:]:
                print("##" ,i.strip())
                print()
            print("---------------------------------------------------")
        return out
    prompt(input,  prnout=True)

#Instruction: แปลภาษาไทยเป็นอังกฤษ 

## Input:
ลดน้ำหนักต้องทำอย่างไร

## Response:
How to Lose Weight

---------------------------------------------------
## Input:
เขียนโปรแกรม python export csv pandas

## Response:
Python programming, export to csv, pandas.

---------------------------------------------------
## Input:
วิธีการทำน้ำจิ้มไก่

## Response:
How to make chicken dipping.

---------------------------------------------------
## Input:
แต่งกลอนวันแม่

## Response:
Write a poem for Mothers Day.

---------------------------------------------------
## Input:
ศึกชิงนางเดือด สามีแทงกิ๊กหนุ่มเสียชีวิตกลางถนนเพชรบุรีตัดใหม่ ก่อนหน้านี้ เคยปะทะคารมกันมาแล้วไม่มีใครยอมใคร

## Response:
A bawdy brawl erupts at a bacchanal, a young man is stabbed to death by his jealous husband at the end of Phaholyothin Road.

---------------------------------------------------
## Input:
กรุงเทพมหานคร เป็นเมืองหลวงและนครที่มีประชากรมากที่สุดของประเทศไทย เป็นศูนย์กลางการปกครอง การศึกษา การคมนาคมขนส่ง กา

## Load from existing dataset Thai Unlabeled

the Thai dataset by Bhunakit as of 17 Nov 2023 consist mainly of 

    - Paraphrase Thai dataset by ChatGPT3.5-Turbo from published Thai documents
        - News Articles
        - History Examination paper

In [7]:
# Load and Print the dataset
MAX_INPUT = 10

dataset = load_dataset("Bhunakit/paraphrasethai")

print(dataset["train"])

for i, block in enumerate(dataset["train"]):
    if i == MAX_INPUT:
        break
    print(f"{i}------------------------------------------")
    # print(block)
    print(block["original"])
    print(f"----------")
    print(block["paraphrased"])
    print(f"----------")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['idx', 'original', 'paraphrased'],
    num_rows: 1951
})
0------------------------------------------
รอกันมา 2 สัปดาห์ บอลสโมสรลีกต่างๆของยุโรปก็กลับมาดวลแข้งกันอีกครั้งในวีกนี้ ช่วงฟีฟ่าเดย์ รอบล่าสุดทิ้งทวนคัดบอลโลก มีประเด็นให้ฮือฮากันหลังจากอิตาลี หนึ่งในทีมอมตะของวงการลูกหนังโลก
----------
เมื่อผ่านไป 2 สัปดาห์แล้ว ฟุตบอลสโมสรในลีกต่างๆของยุโรปก็กลับมาระลึกถึงการแข่งขันกันอีกครั้งในวีกนี้ ในช่วงเดือนธันวาคม ที่ผ่านมา มีเรื่องราวที่น่าสนใจหลังจากทีมชาติอิตาลี หนึ่งในทีมอยุทธสัมพันธ์ของโลก
----------
1------------------------------------------
รองผู้ว่าราชการจังหวัดภูเก็ตลงพื้นที่ตรวจสอบเกาะนาคาน้อย ภายหลังที่ ครอบครัว หิรัญพฤกษ์ ออกมาระบุว่า มีกลุ่มชายฉกรรจ์เตรียมเข้ามาแผ้วถางพื้นที่บนเกาะ และข่มขู่ 
----------
พระองค์ในฐานะผู้ว่าราชการจังหวัดภูเก็ตจะออกไปตรวจสอบเกาะนาคาน้อย หลังจากที่ครอบครัวหิรัญพฤกษ์ว่ามีกลุ่มชายกำลังเตรียมเข้ามาล่าศัตรูบนเกาะและข่มขู่
----------
2------------------------------------------
พาณิชย์ ยืนเป้าส่งออกปีนี้ยังเติบโต 5% ไว้เป็นกำ

### Test Prompt

In [8]:
MAX_PROMPT = 5

for i, block in enumerate(dataset["train"]):
    if i == MAX_PROMPT:
        break
    out = prompt(input= block["original"], prnout=True)

## Input:
รอกันมา 2 สัปดาห์ บอลสโมสรลีกต่างๆของยุโรปก็กลับมาดวลแข้งกันอีกครั้งในวีกนี้ ช่วงฟีฟ่าเดย์ รอบล่าสุดทิ้งทวนคัดบอลโลก มีประเด็นให้ฮือฮากันหลังจากอิตาลี หนึ่งในทีมอมตะของวงการลูกหนังโลก

## Response:
Soccer club leagues of Europe are back at war with each other in this Vigon. The last FIFA day, the last selection for the World Cup, has a lot to talk about. After Italy, one of the world's soccer giants, was eliminated.

---------------------------------------------------
## Input:
รองผู้ว่าราชการจังหวัดภูเก็ตลงพื้นที่ตรวจสอบเกาะนาคาน้อย ภายหลังที่ ครอบครัว หิรัญพฤกษ์ ออกมาระบุว่า มีกลุ่มชายฉกรรจ์เตรียมเข้ามาแผ้วถางพื้นที่บนเกาะ และข่มขู่

## Response:
A deputy governor of Phuket inspected the island after the Harnarin family said a group of men was on its way to clear the land and intimidate them.

---------------------------------------------------
## Input:
พาณิชย์ ยืนเป้าส่งออกปีนี้ยังเติบโต 5% ไว้เป็นกำลังใจในการทำงาน แม้เศรษฐกิจโลกจะชะลอตัว จ่อปรับแผนส่งออกใหม่ หวังเจาะตลาด

### Acutal Prompting and exporting the sythetic dataset for Fine-Tuning

In [10]:
not_stupid_list = []
import random
for i, block in enumerate(dataset["train"]):
    not_stupid_list.append([block["original"], block["paraphrased"]])
random.shuffle(not_stupid_list)
print(len(not_stupid_list))
print(not_stupid_list[:10])



1951
[['ทางการศรีลังกาเชื่อว่าเหตุระเบิดที่เกิดขึ้นทั้งหมดมีความเกี่ยวพันกับเครือข่ายก่อการร้ายต่างชาติ ขณะนี้มีรายงานผู้เสียชีวิตมากถึง 290 คน บาดเจ็บอีก 500 คน ', 'เจ้าหน้าที่ศรีลังกาเชื่อว่าการระเบิดที่เกิดขึ้นทั้งหมดเกี่ยวข้องกับเครือข่ายก่อการร้ายต่างชาติ ในขณะนี้รายงานว่าหนึ่งในผู้เสียชีวิตเป็นจำนวน 290 คนและผู้บาดเจ็บอีก 500 คน'], ['คณะเจ้าหน้าที่เก็บกู้วัตถุระเบิดในเยอรมนีประสบความสำเร็จในการเก็บกู้ระเบิดขนาดใหญ่ในสมัยสงครามโลกครั้งที่ 2 ในนครแฟรงก์เฟิร์ต หลังจากพบระเบิดลูกนี้ เมื่อวันพุธที่ 30 สิงหาคมที่ผ่านมา', 'คณะทีมผู้บังคับบัญชาการสำนักงานฝ่ายกู้ภัยวัตถุระเบิดในประเทศเยอรมันประสบความสำเร็จในการสกัดกู้ระเบิดขนาดใหญ่ในช่วงสงครามโลกครั้งที่สองที่เกิดขึ้นในเมืองแฟรงก์เฟิร์ต หลังจากค้นพบวงจรระเบิดรุนแรงนี้ในวันพุธที่ 30 สิงหาคมที่ผ่านมา'], ['ตำรวจพังงา แถลงจับผู้ต้องหาคดีเสพยา-ลักทรัพย์ หลังสะเดาะกุญแจห้องขัง สภ.ตะกั่วป่า หลบหนี ตามล่าตัวเจอบนรถสิบล้อเก่าในสวนปาล์ม รับใช้ลวดแค่เส้นเดียว บอกได้วิชาจากเพื่อนในคุก', 'ตำรวจในพังงาได้แถลงข่าวว่าจับผู้ต้องหาที่มีแนวคดีเกี่ยวกับการใช

In [11]:
# MAX_PROMPT = 2
import pandas as pd
import random
instruction = "แปลภาษาไทยเป็นอังกฤษ"
dataset_instruction = "แปลอังกฤษเป็นภาษาไทย"


# Create an empty DataFrame with specified columns
out_df = pd.DataFrame(columns=['input', 'output', 'instruction'])


for i, ori_para in enumerate(not_stupid_list):
    # if i == MAX_PROMPT:
    #     break
    print(f"prompting_{i}..")
    out = prompt(input= ori_para[0])
    in_res = out.split("###")[-2:]
    # print(f"{=}")
    english_syth = in_res[1].split(":\n")[-1].strip()
    # print(f"{input=}")
    # print(f"{output=}")

    # Data to append
    data_to_append = pd.DataFrame([
        {'input': english_syth, 'output': ori_para[0], 'instruction': dataset_instruction},
        {'input': english_syth, 'output': ori_para[1], 'instruction': dataset_instruction},
                                  ])

    # Append data using concat
    out_df = pd.concat([out_df, data_to_append], ignore_index=True)
out_df
    


prompting_0..
prompting_1..
prompting_2..
prompting_3..
prompting_4..
prompting_5..
prompting_6..
prompting_7..
prompting_8..
prompting_9..
prompting_10..
prompting_11..
prompting_12..
prompting_13..
prompting_14..
prompting_15..
prompting_16..
prompting_17..
prompting_18..
prompting_19..
prompting_20..
prompting_21..
prompting_22..
prompting_23..
prompting_24..
prompting_25..
prompting_26..
prompting_27..
prompting_28..
prompting_29..
prompting_30..
prompting_31..
prompting_32..
prompting_33..
prompting_34..
prompting_35..
prompting_36..
prompting_37..
prompting_38..
prompting_39..
prompting_40..
prompting_41..
prompting_42..
prompting_43..
prompting_44..
prompting_45..
prompting_46..
prompting_47..
prompting_48..
prompting_49..
prompting_50..
prompting_51..
prompting_52..
prompting_53..
prompting_54..
prompting_55..
prompting_56..
prompting_57..
prompting_58..
prompting_59..
prompting_60..
prompting_61..
prompting_62..
prompting_63..
prompting_64..
prompting_65..
prompting_66..
promp

,input,output,instruction
0,Sri Lankan officials believe all of the bombs ...,ทางการศรีลังกาเชื่อว่าเหตุระเบิดที่เกิดขึ้นทั้...,แปลภาษาไทยเป็นอังกฤษ
1,Sri Lankan officials believe all of the bombs ...,เจ้าหน้าที่ศรีลังกาเชื่อว่าการระเบิดที่เกิดขึ้...,แปลภาษาไทยเป็นอังกฤษ
2,A German bomb squad has recovered a large Worl...,คณะเจ้าหน้าที่เก็บกู้วัตถุระเบิดในเยอรมนีประสบ...,แปลภาษาไทยเป็นอังกฤษ
3,A German bomb squad has recovered a large Worl...,คณะทีมผู้บังคับบัญชาการสำนักงานฝ่ายกู้ภัยวัตถุ...,แปลภาษาไทยเป็นอังกฤษ
4,Krabi police have arrested a suspect in a drug...,ตำรวจพังงา แถลงจับผู้ต้องหาคดีเสพยา-ลักทรัพย์ ...,แปลภาษาไทยเป็นอังกฤษ
...,...,...,...
3897,The road network under the supervision of the ...,สายพานิชย์ทางเรียงคามสำหรับการขนส่งทางบกภายใต้...,แปลภาษาไทยเป็นอังกฤษ
3898,Authorities are investigating the human body p...,เจ้าหน้าที่เร่งตรวจสอบอวัยวะมนุษย์ที่พบตามจุดต...,แปลภาษาไทยเป็นอังกฤษ
3899,Authorities are investigating the human body p...,เจ้าหน้าที่ตรวจสอบตำแหน่งที่พบบุคคลในแม่น้ำเจ้...,แปลภาษาไทยเป็นอังกฤษ
3900,"Based on the success of the 5-part drama ""Sara...",จากกระแสความร้อนแรงของละครโทรทัศน์ชุดสุภาพบุรุ...,แปลภาษาไทยเป็นอังกฤษ


In [31]:
# Ensuring the instruction is correct
dataset_instruction = "แปลอังกฤษเป็นภาษาไทย"

out_df["instruction"] = dataset_instruction
out_df

,input,output,instruction
0,Sri Lankan officials believe all of the bombs ...,ทางการศรีลังกาเชื่อว่าเหตุระเบิดที่เกิดขึ้นทั้...,แปลอังกฤษเป็นภาษาไทย
1,Sri Lankan officials believe all of the bombs ...,เจ้าหน้าที่ศรีลังกาเชื่อว่าการระเบิดที่เกิดขึ้...,แปลอังกฤษเป็นภาษาไทย
2,A German bomb squad has recovered a large Worl...,คณะเจ้าหน้าที่เก็บกู้วัตถุระเบิดในเยอรมนีประสบ...,แปลอังกฤษเป็นภาษาไทย
3,A German bomb squad has recovered a large Worl...,คณะทีมผู้บังคับบัญชาการสำนักงานฝ่ายกู้ภัยวัตถุ...,แปลอังกฤษเป็นภาษาไทย
4,Krabi police have arrested a suspect in a drug...,ตำรวจพังงา แถลงจับผู้ต้องหาคดีเสพยา-ลักทรัพย์ ...,แปลอังกฤษเป็นภาษาไทย
...,...,...,...
3897,The road network under the supervision of the ...,สายพานิชย์ทางเรียงคามสำหรับการขนส่งทางบกภายใต้...,แปลอังกฤษเป็นภาษาไทย
3898,Authorities are investigating the human body p...,เจ้าหน้าที่เร่งตรวจสอบอวัยวะมนุษย์ที่พบตามจุดต...,แปลอังกฤษเป็นภาษาไทย
3899,Authorities are investigating the human body p...,เจ้าหน้าที่ตรวจสอบตำแหน่งที่พบบุคคลในแม่น้ำเจ้...,แปลอังกฤษเป็นภาษาไทย
3900,"Based on the success of the 5-part drama ""Sara...",จากกระแสความร้อนแรงของละครโทรทัศน์ชุดสุภาพบุรุ...,แปลอังกฤษเป็นภาษาไทย


In [33]:
with open("out_synthetic_pairs2.csv","w", encoding='utf-8') as file:
    file.write(out_df.to_csv(index=False))

In [41]:
import re

def remove_specific_chars(s):
    s = s.replace('.', ' ')
    s = s.replace('..', ' ')
    s = s.replace('-', ' ')
    s = s.replace('--', ' ')
    s = s.replace('_', ' ')
    return s

# # Example usage
# text = "This is a test string with . and .. and - and --"
# clean_text = remove_specific_chars(text)
# print(clean_text)


with open("out_synthetic_pairs2.csv","r", encoding='utf-8') as file:
    list_csv = file.readlines()
    x = ""
    lil_x = ""
    for i, line in enumerate(list_csv):
        if i < 20:
            print("before: ",line)
        append = remove_specific_chars(line)
        x+= append
        if i < 20:
            print("after: ", append)
            print()
        if i < 300:
            lil_x+= append
            

with open("out_synthetic_pairs2_clean.csv","w", encoding='utf-8') as file:
    file.write(x)

with open("out_synthetic_pairs2_clean_lil.csv","w", encoding='utf-8') as file:
    file.write(lil_x)

before:  input,output,instruction

after:  input,output,instruction


before:  "Sri Lankan officials believe all of the bombs are linked to a foreign terrorist network. At present, there are at least 290 dead, and",ทางการศรีลังกาเชื่อว่าเหตุระเบิดที่เกิดขึ้นทั้งหมดมีความเกี่ยวพันกับเครือข่ายก่อการร้ายต่างชาติ ขณะนี้มีรายงานผู้เสียชีวิตมากถึง 290 คน บาดเจ็บอีก 500 คน ,แปลอังกฤษเป็นภาษาไทย

after:  "Sri Lankan officials believe all of the bombs are linked to a foreign terrorist network  At present, there are at least 290 dead, and",ทางการศรีลังกาเชื่อว่าเหตุระเบิดที่เกิดขึ้นทั้งหมดมีความเกี่ยวพันกับเครือข่ายก่อการร้ายต่างชาติ ขณะนี้มีรายงานผู้เสียชีวิตมากถึง 290 คน บาดเจ็บอีก 500 คน ,แปลอังกฤษเป็นภาษาไทย


before:  "Sri Lankan officials believe all of the bombs are linked to a foreign terrorist network. At present, there are at least 290 dead, and",เจ้าหน้าที่ศรีลังกาเชื่อว่าการระเบิดที่เกิดขึ้นทั้งหมดเกี่ยวข้องกับเครือข่ายก่อการร้ายต่างชาติ ในขณะนี้รายงานว่าหนึ่งในผู้เสียชีวิตเป็นจำนวน 